In [5]:
# Open the company idx file
index_file = open("company.idx").readlines()

#Just confirming the header of the file		
print(index_file[:10])

['Description:           Master Index of EDGAR Dissemination Feed by Company Name\n', 'Last Data Received:    March 31, 2021\n', 'Comments:              webmaster@sec.gov\n', 'Anonymous FTP:         ftp://ftp.sec.gov/edgar/\n', ' \n', ' \n', ' \n', ' \n', 'Company Name                                                  Form Type   CIK         Date Filed  File Name\n', '---------------------------------------------------------------------------------------------------------------------------------------------\n']


In [26]:
print(len(index_file))

361176


In [27]:
# Keep the information
find_list = []
item = 0
line = 0

# while item < 10000:
while line<len(index_file):
  i = index_file[line]
  loc1 = i.find('10-K')
  loc2 = i.find("NT 10-K") 
  loc3 = i.find("10-K/A")

#We strictly keep 10-K files, not NT 10-K or 10-K/A
  if (loc2 == -1) and (loc1 != -1) and (loc3 == -1):
    find_list.append(i)
  line+=1
  item = len(find_list)


print(len(find_list))

5527


In [31]:
# The commands below will split the line, and send the links to a list called ReportList, and the CIK+date issued to a list called
# Company_No (this will be  the names of our files when we download them)
ReportList = []
Company_No = []
Sticker = []
for i in find_list:
    split_i = i.split()
    print(split_i)
    ReportList.append("https://www.sec.gov/Archives/" + split_i[-1])
#     Company_No.append(split_i[-3] + "_" + split_i[-2])
    Company_No.append(split_i[-3])
    Sticker.append(split_i[0])

print(ReportList[:5])
print(Company_No[:5])
print(Sticker)

['10X', 'Capital', 'Venture', 'Acquisition', 'Corp', '10-K', '1821595', '2021-03-30', 'edgar/data/1821595/0001213900-21-018726.txt']
['10x', 'Genomics,', 'Inc.', '10-K', '1770787', '2021-02-26', 'edgar/data/1770787/0001770787-21-000009.txt']
['1847', 'Goedeker', 'Inc.', '10-K', '1810140', '2021-03-29', 'edgar/data/1810140/0001213900-21-018284.txt']
['1895', 'Bancorp', 'of', 'Wisconsin,', 'Inc.', '10-K', '1751692', '2021-03-31', 'edgar/data/1751692/0001193125-21-101902.txt']
['1Life', 'Healthcare', 'Inc', '10-K', '1404123', '2021-03-17', 'edgar/data/1404123/0001564590-21-013897.txt']
['1ST', 'CONSTITUTION', 'BANCORP', '10-K', '1141807', '2021-03-15', 'edgar/data/1141807/0001141807-21-000005.txt']
['1ST', 'SOURCE', 'CORP', '10-K', '34782', '2021-02-18', 'edgar/data/34782/0000034782-21-000036.txt']
['1st', 'FRANKLIN', 'FINANCIAL', 'CORP', '10-K', '38723', '2021-03-30', 'edgar/data/38723/0001376474-21-000102.txt']
['22nd', 'Century', 'Group,', 'Inc.', '10-K', '1347858', '2021-03-11', 'edga

In [40]:
a=[]
for i in range(len(Sticker)):
    if Sticker[i] == 'EBAY':
      a.append(i)  
# len(Sticker)
a

[1667]

In [41]:
for i in a:
#     Sticker[454]
    print(ReportList[i])

https://www.sec.gov/Archives/edgar/data/1065088/0001065088-21-000006.txt


In [11]:
!pwd

/content


In [12]:
import os
os.chdir("/content/10K Files")

def createfile(filename, content):
    name= filename + ".txt"  # Here we define the name of the file
    with open(name, "w") as file:
        file.write(str(content)) # Here define its content, which will be the textual content from the 10-K files.
        file.close()
        print("Succeed!")

In [ ]:
import requests
company_order = 0
unable_request = 0

for a_index in range(100):
    web_add = ReportList[a_index]
    filename = Company_No[a_index]

    webpage_response = requests.get(web_add, headers={'User-Agent': 'Mozilla/5.0'}) 
    # It is very important to use the header, otherwise the SEC will block the requests after the first 5.

    if webpage_response.status_code == 200: 
        # The HTTP 200 OK success status response code indicates that the request has succeeded. 
        body = webpage_response.content
        createfile(filename, body)
    else:
        print ("Unable to get response with Code : %d " % (webpage_response.status_code))
        unable_request += 1

    a_index +=1

print(unable_request) 

In [14]:
# First, create a list with the name of the files in the folder where we downlaoded the 10-K files.
os.chdir("/content/10K Files")
all_files = os.listdir()

# For the purpose of this exercise, I will use only 10 random files from the 100 we downloaded.
import random

# Here we basically create a list with 10 random indexes between 0 and 100 to selected the files in the folder.
random_index = []
while len(random_index) < 10:
    n = random.randint(0,100)
    if n not in random_index:
        random_index.append(n)

# Here we create a new list with the 10 chosen files.
chosen_file = [all_files[i] for i in random_index]
print(chosen_file)

['1721781_2021-02-25.txt', '917251_2021-02-18.txt', '1555279_2021-03-31.txt', '1810140_2021-03-29.txt', '1820191_2021-03-31.txt', '1158449_2021-02-22.txt', '1775098_2021-03-26.txt', '1819438_2021-03-31.txt', '1751692_2021-03-31.txt', '38723_2021-03-30.txt']


In [15]:
def read_txt(file_name):
    txt_file = open(file_name,"r",encoding='UTF8')                                       
    str_txt = txt_file.read()
    return str_txt

In [16]:
# We will use the regex module to get everything between these patterns: <DOCUMENT>\n<TYPE>10-K and </DOCUMENT>
# Using the regex modules

import re
text_start_pattern = re.compile(r'<DOCUMENT>') 
text_end_pattern = re.compile(r'</DOCUMENT>')
type_pattern = re.compile(r'<TYPE>10-K[^\n]+')

# Here we will define a function that will be used to extract the textual data from 10-K txt files.

def textual_content(file):
    doc_start_list = [x.start() for x in text_start_pattern.finditer(file)] #assigns the first index from the starting pattern created before
    doc_end_list = [x.end() for x in text_end_pattern.finditer(file)] #assigns the last index from the ending pattern created before
    type_list = [x[len('<TYPE>'):] for x in type_pattern.findall(file)] #assigns the type of the documents, which will always be 10-K's because we restricted it before

    for doc_type, start_index, end_index in zip(type_list, doc_start_list, doc_end_list):
            report_content = file[start_index:end_index]
    return report_content

In [17]:
text_initial = read_txt(chosen_file[0])
text_10k = textual_content(text_initial)
print(text_10k[0:30])
print(text_10k[-30:])

<DOCUMENT>\n<TYPE>10-K\n<SEQUE
n</XBRL>\n</TEXT>\n</DOCUMENT>


In [18]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup
def BeautifulSoup_clean1(str_txt):
    soup = BeautifulSoup(str_txt,'html.parser')
    return soup.get_text() # Here we return only the textual content, without the XML tags

In [19]:
import string
from nltk.tokenize import word_tokenize

def further_clean(text):

    # First, for each of the following characters (or symbols), the function replaces it by an empty space on the text 
    for a_sign in ['\\n', '\\t', '☐', '☒', '\xa0', '●', '“', '”']:
        text = text.replace(a_sign," ")

    # Here, for each punctution in a set of all existing punctuations, the function also replaces it by an empty space.
    for a_punc in string.punctuation:
        text = text.replace(a_punc, " ")

    # Morever, the fuction replaces '\s+' (which represents a sequence of empty spaces) by an single empty space, avoiding unecessary spaces 
    # and also sets all letters to lower case to make it easier to analyse later.

    text = re.sub('\s+'," ", text).lower()

    return text.strip() 

In [20]:
# Very important: This function here is for exposition only. It will be modified later in section 5 when we start using
# the dictionary of negative words.

# Tokenizing and counting the total frequency of words

from nltk.tokenize import word_tokenize

def word_count(text):
    word_list = word_tokenize(text) #splits the text into words and assigns it to a new list
    total_num = len(word_list) #count the number of words in the new list
    return total_num, word_list

In [31]:
import pandas as pd 

LM_negw = pd.read_csv("/content/Loughran-McDonald_MasterDictionary_1993-2021.csv")

# Transform the file into a list of negative words, and convert all entries to lowercase.
LM_negw = list(LM_negw['Negative'])
LM_negw = [str(j).lower() for j in LM_negw]
# LM_negw = [j.lower() for j in LM_negw if not isinstance(j, int)]

# Since we will be counting negative words, 
# lets create a dictionary that assigns the number of appearances to each negative words

negw_dic = {} #creates an empty dictionary
for A_NegWord in LM_negw:
    negw_dic[A_NegWord] = 0 
# For each word in the LM_negw list, we create an entry inside the dictonary with the word and the number 0.

In [32]:
from nltk.tokenize import word_tokenize

def word_count(text):

    # Tokenizing and counting total words
    word_list = word_tokenize(text) # Splits the text into words and assigns it to a new list
    total_num = len(word_list) # Count the number of words in the new list


    # Counting negative words

    negw_dic_ex = negw_dic.copy() # This commands creates a copy of the dictionary we created with the L&M negative word list

    # This loops checks if each word in the text is a negative word, and if so, adds 1 to the respective entry in 
    # the "negw_dic_ex" dictionary.

    for a_word in word_list:
        if a_word in LM_negw:
            negw_dic_ex[a_word] += 1


    # This commands creates a new variable from a dictionary (the negw_dic_ex created before), and the orient="index" means that
    # the keys of the dictionary will be the rows of the variable.

    negw_df = pd.DataFrame.from_dict(negw_dic_ex, orient='index')
    negw_df.reset_index(inplace=True) # This command modifies the indexes of the list, converting them to default indexes.
    # This command renames the columns of the variable (inplace=True means that the variable will be modified without creating a new variable)
    negw_df.rename(columns={"index": "Negative_words", 0: "Word_counts"}, inplace=True)


    total_neg = sum(list(negw_dic_ex.values()))  # Total number of negative words
    negw_fre = total_neg/total_num # % of negative words in the text

    return {"Negative_WordCount_df":negw_df, "Total_WordNumber":total_num, "Total_Negative_WordCount": total_neg,
        "Negative_WordFrequency": negw_fre}

In [33]:
import nltk
nltk.download('punkt')
# First, we open 4 empty lists to append the 4 returns from the function we've just defined.
Neg_WordDf_list = []
Total_Word_list = []
Total_Neg_Word_list = []
Frequency_list = []

# For each file in the chosen_file list (the list with the 10-K txt files)
for a_file in chosen_file:
    text = read_txt(a_file) #run the read_txt fuction that opens the txt file
    text = textual_content(text) #run the textual_content function to get the textual content from the file opened before.
    text = BeautifulSoup_clean1(text) #run the BeautifulSoup_clean1 function to clean the data even better using the html.parser algorithm. 
    text = further_clean(text) #run the further_clean function to remove symbols, punctiations, and convert strings to lower case. 
    result = word_count(text) #run the word_count functions that tokenize the text, and deal with negative word counting.

    Neg_WordDf_list.append(result['Negative_WordCount_df'])
    Total_Word_list.append(result["Total_WordNumber"])
    Total_Neg_Word_list.append(result["Total_Negative_WordCount"])
    Frequency_list.append(result["Negative_WordFrequency"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# Here we create a new dictionary using data from the lists defined before.
summary = {'File_name':chosen_file, 'Total_WordNumber':Total_Word_list, 'Total_Negative_WordCount':Total_Neg_Word_list,'Negative_WordFrequency':Frequency_list}

summary_df = pd.DataFrame(summary, index=range(1,11))
summary_df

,File_name,Total_WordNumber,Total_Negative_WordCount,Negative_WordFrequency
1,1721781_2021-02-25.txt,215305,2176,0.010107
2,917251_2021-02-18.txt,86784,209,0.002408
3,1555279_2021-03-31.txt,72788,137,0.001882
4,1810140_2021-03-29.txt,62102,59,0.000950
5,1820191_2021-03-31.txt,67005,43,0.000642
6,1158449_2021-02-22.txt,36285,128,0.003528
7,1775098_2021-03-26.txt,3410,8,0.002346
8,1819438_2021-03-31.txt,67418,57,0.000845
9,1751692_2021-03-31.txt,61412,207,0.003371
10,38723_2021-03-30.txt,17084,20,0.001171
